In [1]:
import pandas as pd
import os
import json
import jsonlines
from tqdm import tqdm
import glob
import yaml
from sqlalchemy import create_engine
from sqlalchemy.sql import text

In [2]:
os.getcwd()

'c:\\Users\\nickl\\Documents\\Berkeley\\W210\\final_project\\therapeutic_accelerator\\dev\\database_work\\data_cleanup'

In [5]:
temp_df = pd.DataFrame()

file = "../../../data/papers_metadata/20240426_070545_00280_f8inb_0a6bea85-47c6-4cee-9031-3ee628632813.json.gz"

with jsonlines.open(file, "r") as reader: 
    count = 0
    for line in tqdm(reader.iter(skip_invalid=True)): 
        if count >= 5: 
            break
        else: 
            temp_df = pd.concat([temp_df,pd.DataFrame([line])]) # reads json, converts to dataframe, preprocess functions and appends results to database
            count += 1
            
temp_df.reset_index(inplace = True, drop = True)

ValueError: 'mode' must be either 'r', 'w', 'a', or 'x'

In [14]:
temp_df = pd.DataFrame()

file = "../../../data/papers_metadata/20240426_070545_00280_f8inb_0a6bea85-47c6-4cee-9031-3ee628632813.json.gz"

with open(file, "rb") as reader: 
    count = 0
    lines = reader.readline()
#     for line in tqdm(reader.iter(skip_invalid=True)): 
#         if count >= 5: 
#             break
#         else: 
#             temp_df = pd.concat([temp_df,pd.DataFrame([line])]) # reads json, converts to dataframe, preprocess functions and appends results to database
#             count += 1
            
# temp_df.reset_index(inplace = True, drop = True)

In [18]:
lines

b'\x1f\x8b\x08\x08S\xd2-f\x02\xff20240426_070545_00280_f8inb_0a6bea85-47c6-4cee-9031-3ee628632813.json\x00\xcc\xbdks\xdc\xd6\x956\xfa}\xaa\xe6?\xa0\xf4\xe1\xc8\xaeBS\xfb~\xf1\xf9p\x8a\x92(\x89\xb2(\xe9%i{\x9c9\xa9\xd4\x06\xb0A\xc2\xeanp\x80n\xc9\xd4[S%K\xce\xd5\xb9x\x92x\xe2$vl\xe72\x89\'\xf1\xc4\x8e\x9d\xc4\xb78U\xd1\xe4;\xf5\x17"\xcbv\x12\xa5\xde\xbfp\xd6\x02\xba)\xa2\x1b\x94\xe8\xa689\xb6Du\x83\xc0\x06\xb0\xf7\xda\xeb\xbe\x9e\x15\xc6y\xb11,\xb3$\xf4O\x0e|\xd1w\xdd,)\xc3a\xd1\r\x07\xd9\xa0\xebC7\x1c\xac\xe7E\x19^\xf4\xfd\xa1\x0f7\x86Q7\x8b\xdd \xcb\xfb\xd5\x01\xb8l\xd3\xbb",|\xea\x0b\xdf\x8f}\x9c\x0f\xfb\x830\xce\x06\xd59\xf5\xb7\xac\x9fv\x87\xbe?\xc8\\w\xe2\x17e\xbe\xe1\xfb.\x8e}Y\x86%K3\xdfM\xca<-\x07\xc3ds\xe7\xad\x06\x9b\x1b\xbe\xdcy q\x03\x1f>\x91\x0f\xf1q\xff\xf1\x1fH(\xa8\xb5T(\x1e\x1e\xfa\xdf\x87\xe7\x8f\x9d9\xfcPp6\xef\xfb08|\xfc\xe8\x99\xf3w\xbe\xcd\x17\xff\x94]\xbc\xf3ui\xfe\xe4\x9d/\xc7\xaaiXL\xe0\xc8\xe1\xf1p\x87\xe1\xf8\xf9a\xb4\xe4\xab\xa3\x943\xc3\xb4\xa0x\xf4\xf8\xb9

In [19]:
lines.decode(encoding='')

LookupError: unknown encoding: UNICODE

Get section headers to split text into parts
|corpusID|authors|section(s)|
|---|---|---|

Don't need source, need text and annotations
- stored in lists with indexes convert to human readable

Get Corpus IDs from DB

In [ ]:
# import psycopg2

# psy_engine = psycopg2.connect(
#     database=config['database']['name'],
#     user=config['database']['user'],
#     password=keys['postgres'],
#     host=config['database']['host'],
#     port='5432'
# )

# cur = psy_engine.cursor()

# conn = sql.connect()

# test = cur.execute("SELECT * FROM attributes;") 

# psy_engine.commit()
# cur.close()
# psy_engine.close()

In [ ]:
# pull content out because that's all I need
test = temp_df.loc[0, 'content']
test.pop('source')

In [ ]:
# turn into dataclass
def convert_text(text, key):
    temp = json.loads(text['annotations'][key])
    results = []
    for i in temp: 
        results.append(test['text'][temp['start']:temp['end']])
    return results

In [ ]:
def get_abstract(text_dict):
    """ """
    ant_index = json.loads(text_dict['annotations']['abstract'])[0]
    ant = test['text'][ant_index['start']:ant_index['end']]
    return ant

In [ ]:
get_abstract(test)

Just need sections

In [ ]:
def get_sections(text, indexes): 
    # Looking at sections
    section = {}
    for i in json.loads(text['annotations']['sectionheader']): 
        section['name'] = text['text'][i['start']:i['end']]
        section['start'] = i['start']
        section['end'] = i['end']
        
    return section

In [ ]:
get_sections(test)

Remove figure and table annotations

Pull relevant texts from DB with right corpusID

In [ ]:
temp_df